In [1]:
# Importing Libraries
import selenium
import pandas as pd
import time
from bs4 import BeautifulSoup

# Importing selenium webdriver 
from selenium import webdriver

# Importing required Exceptions which needs to handled
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException

#Importing requests
import requests

# importing regex
import re

In [2]:
# Activating the chrome browser
driver = webdriver.Chrome(r'C:/Desktop/vinay/chromedriver')

In [22]:
# Opening the homepage of Amazon.in
driver.get('https://www.amazon.in/')
# Asking the user to input the keywords he/she wants to search
user_inp = input('Enter the product you want to search : ')

Enter the product you want to search : mouse


In [23]:
search_bar = driver.find_element_by_id("twotabsearchtextbox")    # Locating searc_bar by id
search_bar.clear()                                               # clearing search_bar
search_bar.send_keys(user_inp)                                   # sending user input to search bar
search_button = driver.find_element_by_xpath('//div[@class="nav-search-submit nav-sprite"]/span/input')       # Locating search_button by xpath
search_button.click()                                                                # Clicking the button to start search

In [24]:
start_page = 0
end_page = 3
urls = []
for page in range(start_page,end_page+1):
    try:
        page_urls = driver.find_elements_by_xpath('//a[@class="a-link-normal s-no-outline"]')
        
        # appending all the urls on current page to urls list
        for url in page_urls:
            url = url.get_attribute('href')     # Scraping the url from webelement
            if url[0:4]=='http':                # Checking if the scraped data is a valid url or not
                urls.append(url)                # Appending the url to urls list
        print("Product urls of page {} has been scraped.".format(page+1))
        
        # Moving to next page
        nxt_button = driver.find_element_by_xpath('//li[@class="a-last"]/a')      # Locating the next_button which is active
        if nxt_button.text == 'Next→':                                            # Checking if the button located is next button
            nxt_button.click()                                                    # Clicking the next button
            time.sleep(5)                                                         # time delay of 5 seconds
        # If the current active button is not next button, the we will check if the next button is inactive or not    
        elif driver.find_element_by_xpath('//li[@class="a-disabled a-last"]/a').text == 'Next→':    
            print("No new pages exist. Breaking the loop")  # Printing message and breakinf loop if we have reached the last page
            break
            
    except StaleElementReferenceException as e:             # Handling StaleElement Exception   
        print("Stale Exception")
        next_page = nxt_button.get_attribute('href')        # Extracting the url of next page
        driver.get(next_page)                               # ReLoading the next page

Product urls of page 1 has been scraped.
Product urls of page 2 has been scraped.
Product urls of page 3 has been scraped.
Product urls of page 4 has been scraped.


In [25]:
prod_dict = {}
prod_dict['Brand']=[]
prod_dict['Name']=[]
prod_dict['Rating']=[]
prod_dict['No. of ratings']=[]
prod_dict['Price']=[]
prod_dict['Return/Exchange']=[]
prod_dict['Expected Delivery']=[] 
prod_dict['Availability']=[]
prod_dict['Other Details']=[]
prod_dict['URL']=[]

In [26]:
for url in urls[:4]:
    driver.get(url)                                                        # Loading the webpage by url
    print("Scraping URL = ", url)
    #time.sleep(2)
    
    try:
        brand = driver.find_element_by_xpath('//a[@id="bylineInfo"]')      # Extracting Brand from xpath
        prod_dict['Brand'].append(brand.text)
    except NoSuchElementException:
        prod_dict['Brand'].append('-')
    
    try:
        name = driver.find_element_by_xpath('//h1[@id="title"]/span')      # Extracting Name from xpath
        prod_dict['Name'].append(name.text)
    except NoSuchElementException:
        prod_dict['Name'].append('-')
    
    try:
        rating = driver.find_element_by_xpath('//span[@id="acrPopover"]')  # Extracting Ratings from xpath
        prod_dict['Rating'].append(rating.get_attribute("title"))
    except NoSuchElementException:
        prod_dict['Rating'].append('-')
    
    try:
        n_rating = driver.find_element_by_xpath('//a[@id="acrCustomerReviewLink"]/span')     # Extracting no. of Ratings from xpath
        prod_dict['No. of ratings'].append(n_rating.text)
    except NoSuchElementException:
        prod_dict['No. of ratings'].append('-')
    
    try:
        price = driver.find_element_by_xpath('//span[@id="priceblock_ourprice"]')            # Extracting Price from xpath
        prod_dict['Price'].append(price.text)
    except NoSuchElementException:
        prod_dict['Price'].append('-')
    try:                                                                                     # Extracting Return/Exchange policy from xpath
        ret = driver.find_element_by_xpath('//div[@data-name="RETURNS_POLICY"]/span/div[2]/a')
        prod_dict['Return/Exchange'].append(ret.text)
    except NoSuchElementException:
        prod_dict['Return/Exchange'].append('-')
    try:
        delivry = driver.find_element_by_xpath('//div[@id="ddmDeliveryMessage"]/b')         # Extracting Expected Delivery from xpath
        prod_dict['Expected Delivery'].append(delivry.text)
    except NoSuchElementException:
        prod_dict['Expected Delivery'].append('-')
    
    try:
        avl = driver.find_element_by_xpath('//div[@id="availability"]/span')                # Extracting Availability from xpath
        prod_dict['Availability'].append(avl.text)
    except NoSuchElementException:
        prod_dict['Availability'].append('-')
    
    try:                                                                                    # Extracting Other Details from xpath
        dtls = driver.find_element_by_xpath('//ul[@class="a-unordered-list a-vertical a-spacing-mini"]')
        prod_dict['Other Details'].append('  ||  '.join(dtls.text.split('\n')))
    except NoSuchElementException:
        prod_dict['Other Details'].append('-')
    
    prod_dict['URL'].append(url)                                                            # Saving url
    time.sleep(2)

Scraping URL =  https://www.amazon.in/gp/slredirect/picassoRedirect.html/ref=pa_sp_atf_aps_sr_pg1_1?ie=UTF8&adId=A085294410XQP79XPOYXD&url=%2FAmazonBasics-Wireless-Mouse-Nano-Receiver%2Fdp%2FB01NADN0Q1%2Fref%3Dsr_1_1_sspa%3Fdchild%3D1%26keywords%3Dmouse%26qid%3D1605693329%26sr%3D8-1-spons%26psc%3D1&qualifier=1605693328&id=8807434880200311&widgetName=sp_atf
Scraping URL =  https://www.amazon.in/gp/slredirect/picassoRedirect.html/ref=pa_sp_atf_aps_sr_pg1_1?ie=UTF8&adId=A014876910SWTBTTV2M73&url=%2FLogitech-Silent-Wireless-Mouse-Charcoal%2Fdp%2FB01M72LILF%2Fref%3Dsr_1_2_sspa%3Fdchild%3D1%26keywords%3Dmouse%26qid%3D1605693329%26sr%3D8-2-spons%26psc%3D1&qualifier=1605693328&id=8807434880200311&widgetName=sp_atf
Scraping URL =  https://www.amazon.in/HP-X1000-Wired-Mouse-Black/dp/B009VCGPSY/ref=sr_1_3?dchild=1&keywords=mouse&qid=1605693329&sr=8-3
Scraping URL =  https://www.amazon.in/Dell-Ms116-275-BBCB-Optical-Mouse/dp/B012DT5U96/ref=sr_1_4?dchild=1&keywords=mouse&qid=1605693329&sr=8-4


In [27]:
prod_df = pd.DataFrame.from_dict(prod_dict)
prod_df

Brand  \
0       Brand: AmazonBasics   
1  Visit the Logitech Store   
2        Visit the HP Store   
3      Visit the Dell Store   

                                                Name              Rating  \
0  AmazonBasics Wireless Mouse with Nano Receiver...  4.2 out of 5 stars   
1  Logitech M221 Wireless Mouse, Silent Buttons, ...  4.5 out of 5 stars   
2                  HP X1000 Wired Mouse (Black/Grey)  4.3 out of 5 stars   
3                           Dell MS116 Optical Mouse  4.5 out of 5 stars   

   No. of ratings       Price      Return/Exchange Expected Delivery  \
0  27,544 ratings  ₹ 1,149.00  10 Days Replacement    Friday, Nov 20   
1   4,049 ratings    ₹ 849.00  10 Days Replacement  Saturday, Nov 21   
2  16,728 ratings    ₹ 349.00  10 Days Replacement    Friday, Nov 20   
3   2,312 ratings    ₹ 307.00  10 Days Replacement    Friday, Nov 20   

  Availability                                      Other Details  \
0    In stock.  Smooth, precise and affordable wireless optica...   
1    In stock.  Ultra-quiet mouse with 90% reduced click sound...   
2    In stock.  3 buttons improve productivity  ||  Optical se...   
3    In stock.  Compatible with practically any system  ||  Pl...   

                                                 URL  
0  https://www.amazon.in/gp/slredirect/picassoRed...  
1  https://www.amazon.in/gp/slredirect/picassoRed...  
2  https://www.amazon.in/HP-X1000-Wired-Mouse-Bla...  
3  https://www.amazon.in/Dell-Ms116-275-BBCB-Opti...

In [ ]:
#saving data to csv
prod_df.to_csv('Amazon_{}.csv'.format(user_inp))